# Quantum Evolution with Measurement and  Reset : Method 2

This method has O(n) qubit complexity making it expensive.

NOTE: Here we have considered wire 0 as signal or feedback qubit, over which we get yi value.

# Initiation

We initiate the environment and load util functions

In [ ]:
! pip install pennylane
! pip install qiskit # This also installs qiskit-ibm-runtime
! pip install pennylane-qiskit

In [2]:

import datetime
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

from functools import partial

dev_q = qml.device("default.qubit", wires=3)
dev = qml.device("default.qubit", wires=3, shots=1024)
IBM=False




@qml.qnode(dev_q)
def matrix_eval(u, wires):
    u(wires)
    return qml.expval(qml.Identity(0))

def get_list(target_string):
    l = list(target_string)
    return [ int(i) for i in l ]

def calculate_state(base, start, expected):
    if len(expected) == 0:
        return start
    start = base @ start
    if expected[0]:
        amp = sum(start[:4])
        start = np.array(  list(start[4:]) + [0]*4) / amp
    else:
        amp = sum(start[4:])
        start = np.array( list(start[0:4]) + [0]*4 ) / amp
    return calculate_state(base, start, expected[1:])

def convert_to_bin(val, length=5):
    num = bin(val)[2:]
    if len(num) <= length:
        num = "0"*(length-len(num)) + num
    else:
       num = num[0:length]
    return num

results = []
def compare(base, target_string, probabilities) -> bool:
        global results
        len_states = len(base)
        actual_prob = calculate_state(base, np.array([1] + [ 0 ]*( len_states -1 ) ), get_list(target_string))

        max_qx = 0
        max_val = 0
        for qc, qq in enumerate(actual_prob):
            if qq < 0:
                qq = qq * -1
            if max_val < qq :
                max_val = qq
                max_qx = qc
        max_qx = convert_to_bin(max_qx, 3)[1:]
        calculated_probabilities = []
        for qx, prob in enumerate(probabilities):
            qp_sum = [0]
            for prob_q in range(len(prob)):
                val_q = convert_to_bin(prob_q, 3 + len(target_string))[1:3]
                if val_q == max_qx:
                    qp_sum.append(prob[prob_q])
            if qp_sum:
              calculated_probabilities.append(sum(qp_sum))
        results.append((len(target_string), actual_prob, probabilities))
        return all(calculated_probabilities)

def get_simulated_state_vectors(unitary, target_string):
    return [ create_circuit(unitary, target_string) ]



def amplitude_damping_unitary_stinespring(gamma):
    U_matrix = np.array([
        [1, 0, 0, 0],                            # |00> -> |00>
        [0, np.sqrt(1 - gamma), np.sqrt(gamma), 0], # |10> -> sqrt(1-g)|10> + sqrt(g)|01>
        [0, -np.sqrt(gamma), np.sqrt(1 - gamma), 0], # |01> -> -sqrt(g)|10> + sqrt(1-g)|01>
        [0, 0, 0, 1]                             # |11> -> |11>
    ])
    return U_matrix

def get_probability(target_string):
  probability =0
  if target_string =="1":
    probability =1
  return probability

/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


# Execute this section if you want to run this on IBM

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Batch



## Please add token
ibm_token =
crn =

QiskitRuntimeService.save_account(channel="ibm_cloud", token=ibm_token, overwrite=True, instance=crn)


# 1. Access your saved IBM Quantum account
# If you saved your account using QiskitRuntimeService.save_account() previously,
# you can just do:
service = QiskitRuntimeService()
try:
    backend = service.least_busy(operational=True, simulator=False, min_num_qubits=2)
    print(f"Using least busy real backend: {backend.name}")
except:
    # Fallback to a simulator if no real hardware is available or if you prefer a cloud sim
    backend = service.get_backend("ibmq_qasm_simulator") # A powerful cloud simulator
    print(f"Using IBM cloud simulator backend: {backend.name}")

# 3. Initialize the PennyLane device with the chosen Qiskit backend
# 'qiskit.remote' is the recommended device for Qiskit Runtime

IBM=True


Using least busy real backend: ibm_torino


In [ ]:
dev = qml.device("qiskit.remote", wires=backend.num_qubits, backend=backend, shots=1024)

# Test Script

Defined different U operations and use them for testing.

In [3]:
def u_1(wires):
  qml.X(wires=wires[1])
  qml.X(wires=wires[0])
  qml.H(wires=wires[0])
  qml.CNOT(wires=[wires[0], wires[2]])
  qml.CNOT(wires=[wires[2], wires[1]])

def u_2(wires):
  qml.Hadamard(wires[0])
  qml.H(wires=2)
  qml.CNOT(wires=wires[0:2])
  qml.X(wires=2)
  qml.H(wires=2)
  qml.CNOT(wires=[wires[1], wires[2]])

def u_3(wires):
  qml.X(wires=1)
  qml.X(wires=0)
  qml.Hadamard(wires[0])
  qml.CNOT(wires=wires[0:2])
  qml.CNOT(wires=[wires[0], wires[2]])

def u_4(wires):
  qml.X(wires=1)
  qml.X(wires=0)
  qml.CNOT(wires=[wires[1], wires[2]])
  qml.Hadamard(wires[0])
  qml.CNOT(wires=wires[0:2])
  qml.CNOT(wires=[wires[0], wires[2]])

def u_5(wires):
  qml.Hadamard(wires[0])
  qml.CNOT(wires=wires[0:2])
  qml.X(wires=2)
  qml.CNOT(wires=[wires[0], wires[2]])
  qml.CNOT(wires=[wires[2], wires[1]])
  qml.CNOT(wires=[wires[1], wires[0]])

def u_6(wires):
  qml.X(wires=wires[1])
  qml.X(wires=wires[0])
  qml.H(wires=wires[0])
  qml.CZ(wires=[wires[0], wires[1]])
  qml.CNOT(wires=[wires[0], wires[2]])
  qml.CNOT(wires=[wires[2], wires[1]])


def u_7(wires):
  qml.H(wires=wires[1])
  qml.CZ(wires=[wires[1], wires[2]])
  qml.H(wires=wires[0])
  qml.H(wires=wires[1])
  qml.CZ(wires=[wires[0], wires[1]])
  qml.CNOT(wires=[wires[0], wires[2]])
  qml.CNOT(wires=[wires[2], wires[1]])

registered_tests = [ u_7 ]


def test(unitary, target_string, debug=0):
  state_vector = qml.matrix(matrix_eval)(unitary, range(3))
  for target in range(len(target_string)):
      if target < len(target_string) - 1: continue
      if debug: print(f"Running tests for n = {target}")
      t1 = datetime.datetime.now()
      assert compare(state_vector, target_string[:target+1], get_simulated_state_vectors(unitary, target_string[:target+1])),\
        f"Error with execution of target string {target_string[:target+1]}"
      if debug: print(f"Took { datetime.datetime.now() - t1 } time to execute for {target_string[:target+1]}")
  print("All test cases passed")


def run_tests(target_string, debug=0):
  for func in registered_tests:
    print(f"Running test for {func.__name__}")
    test(func, target_string, debug=debug)


# Logic

We apply U followed by amplify and reset block (ARB). Main function is create_circuir which calls evolution_measurement_and_reset and executes the U and grover block. We pass in the U operator and target string which we need to simulate without measurement.

In [4]:



num_wires = 3
num_qubits = 3

#simulator_type = "qiskit.remote" #"default.mixed"


def initial_state_preparation(unitary, wires):
    for i in range(3, len(wires)):
        unitary(wires)
        qml.CNOT(wires=[0,i])

def get_num_of_iteration(target_string):
  return int((len(target_string)**(1/2)* np.pi / 4))

def grover_oracle(wires, target_string):
  """
  Oracle to mark the target_string with a phase flip.
  Assumes target_string is a binary string, e.g., "01".
  """
  n_qubits = len(wires)
  if len(target_string) != n_qubits:
      raise ValueError("Target string length must match number of wires.")

  # Apply X gates to map target_string to |11...1>
  for i, bit in enumerate(target_string):
      if bit == '0':
          qml.PauliX(wires=wires[i])

  # Apply Z gate if there's only one qubit
  if n_qubits == 1:
      qml.PauliZ(wires=wires[0])
  # Apply controlled-Z for 2 qubits
  elif n_qubits == 2:
      qml.CZ(wires=[wires[0], wires[1]])
  else:
      qml.H(wires=wires[-1])
      qml.MultiControlledX(wires=wires) # Acts on all wires, last is target
      qml.H(wires=wires[-1])

  for i, bit in enumerate(target_string):
      if bit == '0':
          qml.PauliX(wires=wires[i])

# --- 3. Reflection about the |0...0> state (S_0) ---
def reflect_about_zero_state(wires):
    """
    Implements 2|0...0><0...0| - I using Pauli-X and multi-controlled Z.
    """
    n_qubits = len(wires)

    for wire in wires:
        qml.PauliX(wires=wire)

    if n_qubits == 1:
        qml.PauliZ(wires=wires[0])
    elif n_qubits == 2:
        qml.CZ(wires=[wires[0], wires[1]])
    else:
        # For more than 2 qubits, use MultiControlledX and H to implement MCZ
        qml.H(wires=wires[-1])
        qml.MultiControlledX(wires=wires) # Acts on all wires, last is target
        qml.H(wires=wires[-1])

    for wire in wires:
        qml.PauliX(wires=wire)

def diffusion(func, target_wires):
  qml.adjoint(func)()
  reflect_about_zero_state(target_wires)
  func()


def evolution_measurement_and_reset(unitary, target_string):
    n_qubits = num_qubits + len(target_string)

    num_iterations = get_num_of_iteration(target_string)

    # Prepare the initial state |A>
    initial_state_preparation(unitary, range(n_qubits))
    target_qubit = range(3, n_qubits)

    # Apply the Amplitude Amplification Operator Q
    for _ in range(num_iterations):
        grover_oracle(target_qubit, target_string)
        diffusion(partial(initial_state_preparation,unitary, range(n_qubits)), target_qubit)


    return qml.probs(wires=range(n_qubits))


## Creating circuits based on the target string size
def create_circuit(unitary, target_string):
  if IBM:
    dev = qml.device("qiskit.remote", wires=num_wires + len(target_string), shots=1024, backend=backend)
  else:
    dev = qml.device("default.mixed", wires=num_wires + len(target_string), shots=1024)
  return qml.qnode(dev)(evolution_measurement_and_reset)(unitary, target_string)


# Run Test

In [5]:
target_string = "0111101"
run_tests(target_string, debug=1)
# target_string = "011010"
# run_tests(target_string)

Running test for u_7
Running tests for n = 6
Took 0:00:37.685809 time to execute for 0111101
All test cases passed
